In [30]:
from agents import Agent,trace,Runner,function_tool, WebSearchTool
from langchain.tools import DuckDuckGoSearchResults
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown
from pydantic import BaseModel
from pprint import pprint
import asyncio


from dotenv import load_dotenv,find_dotenv
import os
load_dotenv(find_dotenv(),override=True)

True

In [2]:
agent = Agent(name="Jokester", instructions="You are a joke teller", model="gpt-4o-mini")

In [3]:
with trace("Telling a joke"):
    result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
    print(result.final_output)

Why did the autonomous AI agent get kicked out of the party?

Because it kept trying to optimize everyone's conversations!


## First Agent : Search Agent

In [22]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size='low')],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)


In [24]:
message = "What are the most popular and successful AI Agent frameworks in July 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))


As of July 2025, several AI agent frameworks have gained prominence for their capabilities in developing sophisticated AI applications:

- **LangChain**: A modular Python framework that enables developers to build applications powered by large language models (LLMs). It supports various LLMs, including OpenAI, Anthropic, and Hugging Face, and offers tools for chaining prompts, managing memory, and integrating external tools. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **AutoGen**: Developed by Microsoft, AutoGen is an open-source framework that facilitates the creation of multi-agent conversations using LLMs. It allows for the orchestration of autonomous or human-in-the-loop AI agents to collaborate on complex tasks. ([lollypop.design](https://lollypop.design/blog/2025/june/ai-agent-frameworks/?utm_source=openai))

- **CrewAI**: An open-source framework designed for building autonomous multi-agent systems, CrewAI assigns specialized roles to each agent, enabling collaboration to complete complex tasks. It is particularly useful in applications like research, content generation, and automation. ([lollypop.design](https://lollypop.design/blog/2025/june/ai-agent-frameworks/?utm_source=openai))

- **OpenAI Agents SDK**: Released in March 2025, this lightweight Python framework focuses on creating multi-agent workflows with comprehensive tracing and guardrails, facilitating the development of complex AI systems. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **LangGraph**: An extension of LangChain, LangGraph allows developers to define agents as state machines, providing fine-grained control over agent state and supporting complex workflows with deterministic control, retries, and branching logic. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

These frameworks are widely adopted for their flexibility, scalability, and support for complex AI agent development, catering to a range of applications from enterprise solutions to research and content generation. 

## Second Agent - Planner Agent

In [26]:
HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)


In [ ]:
message = "What are the most popular and successful AI Agent frameworks in July 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)


WebSearchPlan(searches=[WebSearchItem(reason='To find the latest frameworks in demand for AI agents by July 2025.', query='popular AI agent frameworks July 2025'), WebSearchItem(reason='To gather insights on the most successful AI agents used in various industries.', query='successful AI agents frameworks 2025'), WebSearchItem(reason='To get reviews and community feedback about AI agent frameworks.', query='AI agent frameworks comparison reviews July 2025'), WebSearchItem(reason='To discover industry reports or analyses on AI frameworks.', query='AI frameworks industry report July 2025'), WebSearchItem(reason='To check for articles discussing advancements in AI agent technologies.', query='advancements in AI agent frameworks 2025')])


## Third Agent - Writer Agent

In [29]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)


### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [31]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### Functions to write a report

In [32]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

In [ ]:
query ="What are the most popular and successful AI Agent frameworks in July 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    print("Hooray!")
display(Markdown(report.markdown_report))